<a href="https://colab.research.google.com/github/har200509/Document_classifier/blob/main/Document_classifier_w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [184]:
# Import necessary libraries
import pandas as pd

In [185]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [186]:
# Import word_tokenize for tokenization
from nltk.tokenize import word_tokenize

In [187]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [188]:
# Import stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [189]:
# Import WordNetLemmatizer for lemmatization
from nltk.stem import WordNetLemmatizer
lemma=WordNetLemmatizer()

In [190]:
# Import string and re for text processing
import string
import re

In [191]:
# Load the dataset
df=pd.read_csv('/content/results.csv')

In [192]:
df.head()

,document,text
0,E WAY BILL,e way bill e way bill e way bill date pm gen...
1,E WAY BILL,e way bill e way bill e way bill date pm gen...
2,E WAY BILL,blank e way bill e way bill e way bill date ...
3,E WAY BILL,e way bill e way bill e way bill date pm gen...
4,E WAY BILL,e way bill e way bill e way bill date pm gen...


In [193]:
# Convert 'document' column to lowercase
df['document'] = df['document'].str.lower()

In [194]:
df['document'].value_counts()

,count
document,
delivery challan,230
e way bill,222
invoice,203
bill of entry,76
purchase order,70
debit note,60
credit note,57


In [195]:
# Convert the 'text' column to a list
corpus_df=df['text'].tolist()

In [196]:
len(corpus_df)

918

In [197]:
# Remove punctuation from the text (This cell has a logical error, replace does not modify in place. This might need to be corrected)
for i in range(len(corpus_df)):
  for punc in string.punctuation:
    if punc in corpus_df[i]:
      corpus_df[i].replace(punc,"")

In [198]:
# Define a regex pattern
pattern=r'^[a-z]{2,3}$'

In [16]:
# Text preprocessing: tokenize, lemmatize, remove stopwords and short words
for i in range(len(corpus_df)):
    words = word_tokenize(corpus_df[i])
    words = [lemma.lemmatize(word.lower()) for word in words if word.lower() not in stopwords.words('english')]
    words = [word for word in words if not re.match(pattern, word)]
    corpus_df[i] = " ".join(words)

In [199]:
corpus_df[0]

'e way bill e way bill  e way bill date  pm generate  aaa pw  l z  j k industries valid  pm  kms  valid  part gstin supplier  aaapw l z  j k industries place dispatch mumbai maharashtra  gstin recipient  aaa cj  n zg  jsw steel limited  village dolvi place delivery village dolvi maharashtra  document  document date  transaction type  regular value goods  hsn code  reason transportation outward supply transporter  aacca c zy  associated road carriers limited part b vehicle trans cewb  multi veh info mode entered date enter doc dt    road mh bw  wadkhal  pm  aacca c zy '

In [200]:
# Split the corpus into a list of words
document_df=[]
for para in corpus_df:
  words=para.split(" ")
  document_df.append(words)

In [211]:
len(document_df)

918

In [212]:
# Import pickle for saving/loading models
import pickle

In [203]:
%pip install gensim

In [204]:
# Import gensim
import gensim

In [205]:
# Import Word2Vec and KeyedVectors
from gensim.models import Word2Vec, KeyedVectors

In [206]:
# Define a class for Word2Vec model training
class Word2vec(object):
    def __init__(self, documents):
        self.documents = documents

    def train_w2v(self):
        # Train the Word2Vec model
        w2v_model = Word2Vec(self.documents, vector_size=200, window=10, min_count=1, alpha=0.03, min_alpha=0.0007, sg=1)

        w2v_model.build_vocab(self.documents, progress_per=10000, update=True)
        w2v_model.train(self.documents, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)

        # Save the model
        with open('w2v_model.bin', 'wb') as b:
            pickle.dump(w2v_model, b, protocol=pickle.HIGHEST_PROTOCOL)

        return w2v_model

In [207]:
import numpy as np

# Function to get average Word2Vec vectors for documents
def get_avg_w2v_vectors(documents, model):
    vector_size = model.vector_size
    avg_vectors = []
    for doc in documents:
        valid_words = [word for word in doc if word in model.wv.key_to_index]
        if valid_words:
            vecs = [model.wv[word] for word in valid_words]
            avg_vec = np.mean(vecs, axis=0)
        else:
            avg_vec = np.zeros(vector_size)
        avg_vectors.append(avg_vec)

    return np.array(avg_vectors)

In [208]:
# Train the Word2Vec model
model=Word2vec(document_df)
model.train_w2v()

In [209]:
# Load the trained Word2Vec model
w2v_mod=gensim.models.Word2Vec.load('w2v_model.bin')

In [213]:
len(w2v_mod.wv.key_to_index)

11539

In [214]:
# Get average Word2Vec vectors for the documents
avg_w2v_df=get_avg_w2v_vectors(document_df,w2v_mod)

In [215]:
len(avg_w2v_df)

918

In [216]:
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [217]:
# Encode the 'document' column
df['document']=le.fit_transform(df['document'])

In [218]:
df['document'].value_counts()

,count
document,
3,230
4,222
5,203
0,76
6,70
2,60
1,57


In [219]:
# Define a dictionary for document labels
doc_label={0:'bill of entry',1:'credit note',2:'debit note',3:'delivery challan',4:'e way bill',5:'invoice',6:'purchase order   '}

In [220]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(avg_w2v_df,df['document'],test_size=0.2,random_state=42,stratify=df['document'])

In [221]:
# Import SVC model and train it
from sklearn.svm import SVC

svc_model=SVC(kernel='linear',C=1)

svc_model.fit(x_train,y_train)

SVC(C=1, kernel='linear')

In [222]:
# Import evaluation metrics
from sklearn.metrics import accuracy_score,confusion_matrix

In [223]:
# Calculate accuracy on the test set
accuracy_score(y_test,svc_model.predict(x_test))

0.9728260869565217

In [224]:
# Calculate accuracy on the training set
accuracy_score(y_train,svc_model.predict(x_train))

0.9836512261580381

In [225]:
# Calculate confusion matrix on the test set
confusion_matrix(y_test,svc_model.predict(x_test))

array([[15,  0,  0,  0,  0,  0,  0],
       [ 0,  7,  1,  0,  0,  3,  0],
       [ 0,  0, 12,  0,  0,  0,  0],
       [ 0,  0,  0, 46,  0,  0,  0],
       [ 0,  0,  0,  0, 45,  0,  0],
       [ 1,  0,  0,  0,  0, 40,  0],
       [ 0,  0,  0,  0,  0,  0, 14]])

In [226]:
!pip install xgboost

In [227]:
# Import XGBoost
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [228]:
# Define best parameters and train the XGBoost model
best_params = {
    'colsample_bytree': 0.7131,
    'gamma': 0.4469,
    'learning_rate': 0.302,
    'max_depth': 5,
    'n_estimators': 257,
    'reg_alpha': 0.814,
    'reg_lambda': 2.744,
    'subsample': 0.774
}

final_model = XGBClassifier(**best_params, objective='multi:softprob', tree_method='hist')
final_model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7131, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.4469, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.302, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=257, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [230]:
# Calculate accuracy on the test set for XGBoost
accuracy_score(y_test,final_model.predict(x_test))

0.967391304347826

In [231]:
# Calculate accuracy on the training set for XGBoost
accuracy_score(y_train,final_model.predict(x_train))

0.9986376021798365

In [232]:
# Calculate confusion matrix on the test set for XGBoost
confusion_matrix(y_test,final_model.predict(x_test))

array([[15,  0,  0,  0,  0,  0,  0],
       [ 0,  6,  1,  0,  0,  4,  0],
       [ 0,  0, 12,  0,  0,  0,  0],
       [ 0,  0,  0, 46,  0,  0,  0],
       [ 0,  0,  0,  0, 45,  0,  0],
       [ 0,  0,  0,  1,  0, 40,  0],
       [ 0,  0,  0,  0,  0,  0, 14]])